In [1]:
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense

In [2]:
test = pd.read_table("/Users/audreyq.fu/Documents/CausalityGenomics/Data/RF_test_data/drugs_test.txt", index_col=0)

In [5]:
# test has 105 columns
# the first 5 columns are drugs with NAs
# the next 100 columns are fitness scores or expressions without NAs
print (test)

         SNX-2112    YM-155  TG-101348  LINSITINIB  GW-843682X     A4GNT  \
X42MGBA  0.280280       NaN   0.886875    0.962013         NaN -0.068142   
X5637    0.405890  0.474511   1.000000    1.000000         NaN -0.054576   
X639V    0.254365  0.031780   0.803758    0.999916         NaN  0.021990   
X647V    0.254854       NaN   0.786826    0.999945         NaN -0.047362   
X769P    0.409305  0.058428   0.713465    0.997755         NaN -0.081948   
...           ...       ...        ...         ...         ...       ...   
VMRCRCW  0.554861  0.516973   0.906079    0.999965         NaN  0.036184   
WM793    0.246605  0.041788   0.875728    0.999999         NaN -0.065263   
YAPC     0.464728  0.209115   0.815977    0.996729         NaN  0.061016   
YH13     0.387641  0.039541   0.868399         NaN         NaN -0.089472   
YKG1     0.232776  0.046698   0.836316    0.999783         NaN -0.080561   

             ABI3   ADAMTS4     AP2A1  ARHGAP22  ...  TANGO2_RPKM  \
X42MGBA -0.140820 

In [6]:
# test the autoencoder on data without NAs
test_features = test.iloc[:, 5:104]

In [7]:
test_features

,A4GNT,ABI3,ADAMTS4,AP2A1,ARHGAP22,ARHGAP27,ATP2A2,BCAT1,C10orf120,C1QC,...,STON1_RPKM,TANGO2_RPKM,TBC1D23_RPKM,TEX261_RPKM,TJP2_RPKM,TRMT11_RPKM,TUBA3FP_RPKM,UCHL5_RPKM,USP38_RPKM,XRCC6_RPKM
X42MGBA,-0.068142,-0.140820,0.098996,-0.026153,-0.106601,-0.159593,-0.251190,-0.037508,-0.064946,-0.086767,...,0.279638,0.809593,1.108446,1.535130,0.862357,0.777145,0.579306,0.760519,0.559275,2.248875
X5637,-0.054576,0.023127,-0.038027,-0.110573,0.005369,-0.010394,0.173227,-0.097730,-0.053981,-0.015120,...,0.074176,0.876066,0.969007,1.827928,1.557037,0.934874,0.075021,0.956842,0.714097,2.387127
X639V,0.021990,0.008136,0.002856,0.158422,-0.028162,-0.036205,-0.472033,-0.008139,-0.066713,-0.054276,...,0.254469,1.096474,1.150022,1.623207,0.845878,1.046042,0.173654,0.965003,0.827450,2.592920
X647V,-0.047362,-0.096831,-0.041601,-0.101500,-0.083581,-0.030016,-0.238018,-0.000502,-0.075381,-0.103981,...,0.097181,0.842384,0.968617,1.697187,1.340662,1.011521,0.119300,1.026124,0.681602,2.254637
X769P,-0.081948,0.097837,-0.041427,0.073528,0.107474,-0.034997,0.111017,0.032460,0.078136,-0.046358,...,0.073445,0.964236,1.050812,1.588263,1.459862,1.013794,0.000206,1.248863,0.730629,2.292813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VMRCRCW,0.036184,-0.009946,-0.128264,-0.037957,0.096579,-0.000063,-0.137113,-0.096636,0.093793,-0.033541,...,0.024021,0.595766,1.319546,1.437091,1.276630,1.130778,0.001049,0.807412,0.645825,2.161809
WM793,-0.065263,0.003473,-0.002140,-0.113852,0.070115,-0.060271,-0.004359,0.051585,-0.048291,-0.017518,...,0.414017,0.904798,1.172961,1.545576,0.953946,0.972780,0.000203,1.126378,0.650043,2.198203
YAPC,0.061016,0.011903,0.034378,-0.142186,0.031097,0.146273,0.176120,0.109587,-0.007617,0.014684,...,0.052132,0.842945,0.811011,1.366177,1.573688,1.118233,0.258357,0.816897,0.669286,2.133671
YH13,-0.089472,0.044958,0.037185,-0.106020,0.084216,-0.071836,-0.241921,0.144326,0.011017,-0.181374,...,0.463714,0.702057,1.050615,1.573924,1.137141,1.039405,0.226621,0.882822,0.807128,2.286804


In [8]:
# Function to set up the autoencoder
#
# Several lines below are commented out.
# Those lines may be used to construct an autoencoder with
# an arbitrary number of layers.
# The number of layers is specified by num_layer.
def DAE(train,num_layer):
    # some specs of the autoencoder
    input_size = train.shape[1]
    print ('input size: ', input_size)
    #num_layer = 2
    theta = 1 #int(input_size/num_layer)
#    code_size = input_size-theta*(num_layer+1)
# the size of the bottleneck layer is hard-coded
    code_size = 10
    epochss=100  
#    lrr=0.01

    # start building the autoencoder
    autoencoder = tf.keras.Sequential()
    # set up the input layer
    autoencoder.add(Dense(input_size, input_shape=(input_size,)))

#    for index in range(num_layer):
#        layer_size=input_size-(index+1)*theta
#        autoencoder.add(Dense(layer_size,input_shape=(input_size,),activation='linear'))
#        print(layer_size)

    # add the middle layer
    # only one middle layer 
    autoencoder.add(Dense(code_size,activation='linear'))
    print('size of the middle layer: ', code_size) 

#    for index in range(num_layer):
#        layer_size=input_size-(num_layer-index)*theta
#        autoencoder.add(Dense(layer_size,input_shape=(input_size,),activation='linear'))
#        print(layer_size)

    # add the output layer
    autoencoder.add(Dense(input_size,activation='linear'))
    
    # specify other arguments
    autoencoder.compile(optimizer='Adam', loss='mean_squared_error', metrics=['accuracy'])

    return autoencoder


In [9]:
# using test_features (no NAs) as the input
autoencoder = DAE(test_features,3)

input size:  99
size of the middle layer:  10


In [10]:
history = autoencoder.fit(test_features,test_features, epochs=1000,validation_split=0.2)

Epoch 1/1000
5/5 [==============================] - 0s 32ms/step - loss: 0.5506 - accuracy: 0.0068 - val_loss: 0.4687 - val_accuracy: 0.1892
Epoch 2/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.4677 - accuracy: 0.0612 - val_loss: 0.4689 - val_accuracy: 0.1351
Epoch 3/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.4693 - accuracy: 0.0680 - val_loss: 0.4564 - val_accuracy: 0.1892
Epoch 4/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.4480 - accuracy: 0.1088 - val_loss: 0.4290 - val_accuracy: 0.4054
Epoch 5/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.4236 - accuracy: 0.1973 - val_loss: 0.4083 - val_accuracy: 0.4054
Epoch 6/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.4016 - accuracy: 0.1973 - val_loss: 0.3818 - val_accuracy: 0.3784
Epoch 7/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.3709 - accuracy: 0.2789 - val_loss: 0.3460 - val_accuracy: 0.5676
Epoch 8/1000
5/5 [=

5/5 [==============================] - 0s 5ms/step - loss: 0.0252 - accuracy: 0.5034 - val_loss: 0.0299 - val_accuracy: 0.5405
Epoch 60/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0252 - accuracy: 0.4966 - val_loss: 0.0298 - val_accuracy: 0.5676
Epoch 61/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0252 - accuracy: 0.4966 - val_loss: 0.0298 - val_accuracy: 0.5135
Epoch 62/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0251 - accuracy: 0.5238 - val_loss: 0.0298 - val_accuracy: 0.6216
Epoch 63/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0251 - accuracy: 0.4898 - val_loss: 0.0295 - val_accuracy: 0.5135
Epoch 64/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0250 - accuracy: 0.4898 - val_loss: 0.0295 - val_accuracy: 0.5405
Epoch 65/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0250 - accuracy: 0.5170 - val_loss: 0.0295 - val_accuracy: 0.5676
Epoch 66/1000
5/5 [========

5/5 [==============================] - 0s 4ms/step - loss: 0.0224 - accuracy: 0.4694 - val_loss: 0.0260 - val_accuracy: 0.5946
Epoch 118/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0223 - accuracy: 0.5102 - val_loss: 0.0259 - val_accuracy: 0.6216
Epoch 119/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0223 - accuracy: 0.5170 - val_loss: 0.0258 - val_accuracy: 0.6216
Epoch 120/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0222 - accuracy: 0.4898 - val_loss: 0.0258 - val_accuracy: 0.6216
Epoch 121/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.5034 - val_loss: 0.0257 - val_accuracy: 0.5946
Epoch 122/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0221 - accuracy: 0.4694 - val_loss: 0.0256 - val_accuracy: 0.6216
Epoch 123/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0221 - accuracy: 0.4694 - val_loss: 0.0256 - val_accuracy: 0.6216
Epoch 124/1000
5/5 [=

Epoch 175/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0198 - accuracy: 0.5170 - val_loss: 0.0228 - val_accuracy: 0.5946
Epoch 176/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0198 - accuracy: 0.5102 - val_loss: 0.0227 - val_accuracy: 0.5946
Epoch 177/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0198 - accuracy: 0.5034 - val_loss: 0.0227 - val_accuracy: 0.5946
Epoch 178/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0197 - accuracy: 0.5102 - val_loss: 0.0227 - val_accuracy: 0.6216
Epoch 179/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0197 - accuracy: 0.5170 - val_loss: 0.0226 - val_accuracy: 0.5946
Epoch 180/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0196 - accuracy: 0.5102 - val_loss: 0.0226 - val_accuracy: 0.6216
Epoch 181/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0196 - accuracy: 0.5034 - val_loss: 0.0226 - val_accuracy: 0.5946
Epoch 

Epoch 233/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0179 - accuracy: 0.5306 - val_loss: 0.0209 - val_accuracy: 0.5135
Epoch 234/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0179 - accuracy: 0.5238 - val_loss: 0.0208 - val_accuracy: 0.5405
Epoch 235/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0179 - accuracy: 0.5374 - val_loss: 0.0208 - val_accuracy: 0.5405
Epoch 236/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0178 - accuracy: 0.5306 - val_loss: 0.0209 - val_accuracy: 0.5405
Epoch 237/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0178 - accuracy: 0.5374 - val_loss: 0.0208 - val_accuracy: 0.5135
Epoch 238/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0178 - accuracy: 0.5374 - val_loss: 0.0208 - val_accuracy: 0.5135
Epoch 239/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0178 - accuracy: 0.5374 - val_loss: 0.0207 - val_accuracy: 0.5135
Epoch 

...

Epoch 755/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0119 - accuracy: 0.6259 - val_loss: 0.0152 - val_accuracy: 0.5135
Epoch 756/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0119 - accuracy: 0.6531 - val_loss: 0.0153 - val_accuracy: 0.4865
Epoch 757/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.6599 - val_loss: 0.0152 - val_accuracy: 0.4865
Epoch 758/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.6599 - val_loss: 0.0152 - val_accuracy: 0.4865
Epoch 759/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.6599 - val_loss: 0.0153 - val_accuracy: 0.5135
Epoch 760/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.6395 - val_loss: 0.0153 - val_accuracy: 0.5135
Epoch 761/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.6395 - val_loss: 0.0152 - val_accuracy: 0.4865
Epoch 

Epoch 813/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0117 - accuracy: 0.6327 - val_loss: 0.0151 - val_accuracy: 0.5135
Epoch 814/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0117 - accuracy: 0.6327 - val_loss: 0.0150 - val_accuracy: 0.5135
Epoch 815/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0117 - accuracy: 0.6531 - val_loss: 0.0150 - val_accuracy: 0.5135
Epoch 816/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0117 - accuracy: 0.6531 - val_loss: 0.0151 - val_accuracy: 0.5135
Epoch 817/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0117 - accuracy: 0.6463 - val_loss: 0.0151 - val_accuracy: 0.5135
Epoch 818/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0116 - accuracy: 0.6463 - val_loss: 0.0150 - val_accuracy: 0.5135
Epoch 819/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0117 - accuracy: 0.6327 - val_loss: 0.0150 - val_accuracy: 0.5135
Epoch 

Epoch 871/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0115 - accuracy: 0.6531 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 872/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0115 - accuracy: 0.6531 - val_loss: 0.0150 - val_accuracy: 0.4865
Epoch 873/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.6531 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 874/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0115 - accuracy: 0.6327 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 875/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.6395 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 876/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.6463 - val_loss: 0.0150 - val_accuracy: 0.4865
Epoch 877/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.6259 - val_loss: 0.0150 - val_accuracy: 0.4865
Epoch 

Epoch 929/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0114 - accuracy: 0.6463 - val_loss: 0.0148 - val_accuracy: 0.5135
Epoch 930/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0114 - accuracy: 0.6667 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 931/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0114 - accuracy: 0.6667 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 932/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.6531 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 933/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0114 - accuracy: 0.6531 - val_loss: 0.0148 - val_accuracy: 0.5135
Epoch 934/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.6667 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 935/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.6327 - val_loss: 0.0149 - val_accuracy: 0.4865
Epoch 

Epoch 987/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6803 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 988/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6735 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 989/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6327 - val_loss: 0.0147 - val_accuracy: 0.5135
Epoch 990/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6667 - val_loss: 0.0148 - val_accuracy: 0.4865
Epoch 991/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6667 - val_loss: 0.0147 - val_accuracy: 0.5135
Epoch 992/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.0113 - accuracy: 0.6531 - val_loss: 0.0147 - val_accuracy: 0.5135
Epoch 993/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0113 - accuracy: 0.6803 - val_loss: 0.0148 - val_accuracy: 0.5135
Epoch 

In [11]:
history.history['loss']

[0.5506203174591064,
 0.4676768481731415,
 0.4692823588848114,
 0.4480196237564087,
 0.42359036207199097,
 0.4015936851501465,
 0.37094447016716003,
 0.3321816623210907,
 0.2892478108406067,
 0.2439497411251068,
 0.20080618560314178,
 0.16262187063694,
 0.12975981831550598,
 0.10277584940195084,
 0.08176957070827484,
 0.0658019408583641,
 0.05310560390353203,
 0.04316859319806099,
 0.03606371954083443,
 0.03159593790769577,
 0.029102331027388573,
 0.028107017278671265,
 0.027822410687804222,
 0.027785181999206543,
 0.027646899223327637,
 0.02750278264284134,
 0.027283912524580956,
 0.027159294113516808,
 0.026990840211510658,
 0.026896364986896515,
 0.026816481724381447,
 0.026708479970693588,
 0.026633093133568764,
 0.026577178388834,
 0.026487553492188454,
 0.02639279142022133,
 0.026364624500274658,
 0.026314914226531982,
 0.02625935710966587,
 0.026162484660744667,
 0.0261241365224123,
 0.0260933768004179,
 0.02603609301149845,
 0.02597329020500183,
 0.02591133676469326,
 0.0258618

In [12]:
history.history['val_loss']

[0.46871864795684814,
 0.4689265489578247,
 0.45644503831863403,
 0.4290083050727844,
 0.40825212001800537,
 0.38179919123649597,
 0.3459673225879669,
 0.30496761202812195,
 0.26064714789390564,
 0.21711821854114532,
 0.1778821498155594,
 0.14383120834827423,
 0.11553280800580978,
 0.09309223294258118,
 0.07591261714696884,
 0.062483083456754684,
 0.05175657570362091,
 0.04387601837515831,
 0.03836516663432121,
 0.03528450056910515,
 0.03359120711684227,
 0.03311385214328766,
 0.032879579812288284,
 0.032892677932977676,
 0.032576654106378555,
 0.03260311484336853,
 0.03224360942840576,
 0.03216223791241646,
 0.03198157623410225,
 0.03204729035496712,
 0.03182430565357208,
 0.03178919106721878,
 0.031724121421575546,
 0.03160783648490906,
 0.03143492713570595,
 0.03140069544315338,
 0.031302694231271744,
 0.031442511826753616,
 0.031152570620179176,
 0.031089765951037407,
 0.03109763190150261,
 0.0309444610029459,
 0.031020106747746468,
 0.030859557911753654,
 0.030750809237360954,
 0.

In [13]:
train_mse = autoencoder.evaluate(test_features, test_features, verbose=0) 

In [14]:
train_mse

[0.011940020136535168, 0.6413043737411499]

In [20]:
# using the entire data matrix (with NAs) as the input
autoencoder2 = DAE(test,3)

input size:  105
size of the middle layer:  10


In [21]:
# output shows that this does not work
history2 = autoencoder2.fit(test,test, epochs=1000,validation_split=0.2)

Epoch 1/1000
5/5 [==============================] - 0s 29ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch

5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 116/1000
5/5 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 117/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 118/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 119/1000
5/5 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 120/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 121/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 1

Epoch 172/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 173/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 174/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 175/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 176/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 177/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 178/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 229/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 230/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 231/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 232/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 233/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 234/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 235/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 286/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 287/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 288/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 289/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 290/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 291/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 292/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

...

Epoch 741/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 742/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 743/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 744/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 745/1000
5/5 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 746/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 747/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 798/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 799/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 800/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 801/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 802/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 803/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 804/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 855/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 856/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 857/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 858/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 859/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 860/1000
5/5 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 861/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 912/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 913/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 914/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 915/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 916/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 917/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 918/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0

Epoch 969/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 970/1000
5/5 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 971/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 972/1000
5/5 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 973/1000
5/5 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 974/1000
5/5 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 975/1000
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0

In [ ]:
# The autoencoder set up this way does not work for imputation yet.
# For imputation, we will need to extract the non missing values
# and allocate some of them to the training set, and others to the validation/test set.
# May need to incorporate the function split__csr_matrix() in file scimpute.py.
# scimpute.py is one of the files in our LATE package
# https://github.com/audreyqyfu/LATE/tree/master/scripts